## Fundamentals of deep learning

**Deep Learning** (DL) is a class of Machine Learning methods that aims at learning feature hierarchies. DL is not  the solution but a useful set of tools for building A.I.

We have lots of successful applications on the following contents, which can be viewed as a hierarchical structure.

- Vision <br>
  pixel -> edge -> texton -> super-pixel -> part -> object
- Text <br>
  character -> word -> NP/VP/... -> clause -> sentence -> story
- Audio <br>
  sample -> spectral band -> formant -> motif -> phone -> word

The hierarchical models usually need to be deep.

We will introduce two main deep learning methods and their implementations.

- Convolutional neural network (CNN)
- Recurrent Neural Network (RNN)		

### 1. A Neural Network

<img src="figs/nn.png" style="zoom:50%">
<div align="center">*Example of a fully connected neural network with 2 hidden layers.*</div>

A neural network is constructed by an input layer, several hidden layers and an output layer. 
An [**activation fuction**](http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks) is followed by each layer, generally there are three types -- sigmoid, tanh and ReLU, here we use ReLU.
They introduce non-linearity, here we use ReLU:

$$f(x)=max(0, x)$$

The ReLU layers accomplish as a local linear approximation. Multiple layers yeild exponential savings in number of parameters, through parameter sharing. It can easily obtain by using TensorFlow (TF): <br>
> `tf.nn.relu(x)`

Let's explicitly write expression of the computational process from input to output, which is called **Forward Propagation**.
- $h_1 = max(0, W_1x + b_1)$
- $h_2 = max(0, W_2h_1 + b_2)$
- ...
- $o = W_nh_{n-1} + b_n$

Correspondingly, in TensorFlow, we use: <br>
> `h_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)`

In the following, we use **softmax** funtion to turn the outputs to the probabilities, which can be write as:
$$p(c_j=1|x) = \frac{e^{o_j}}{\sum_{i=1}^{c}e^{o_i}}$$

We now have the probability vector as the output. Since we already have the labelled vector $y$, we can use a [**Loss function**](http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/) (or **cost** function) to measure the distance between these two vectors.
Here we introduce a particular loss function called **Cross entropy**.
$$L(x, y; \theta) = - \sum_j y_j \text{log} p(c_j|x)$$
The above two steps are combined into the same code in TF
> `tf.nn.softmax_cross_entropy_with_logits(o, y)`

To prevent overfitting, we introduce a regularization term (also called a weight decay term) into the loss function, which tends to decrease the magnitude of the weights.

The training (learning) process mainly involve in minimizing the loss function, which use another important method, [**back propagation**](http://neuralnetworksanddeeplearning.com/chap2.html).
Back propagation is about understanding how changing the weights and biases in a network changes the loss function.
The back propagation is based on the chain rule,
- $\frac{\partial L}{\partial W_n} = \frac{\partial L}{\partial o}\frac{\partial o}{\partial W_n}$, $\frac{\partial L}{\partial h_{n-1}} = \frac{\partial L}{\partial o}\frac{\partial o}{\partial h_{n-1}}$
- ...
- $\frac{\partial L}{\partial W_2} = \frac{\partial L}{\partial h_2}\frac{\partial h_2}{\partial W_2}$, $\frac{\partial L}{\partial h_1} = \frac{\partial L}{\partial h_2}\frac{\partial h_2}{\partial h_1}$
- $\frac{\partial L}{\partial W_1} = \frac{\partial L}{\partial h_1}\frac{\partial h_1}{\partial W_1}$

Follow this procedure, we can compute $\partial L/\partial W_i$ and $\partial L/\partial b_i$.

We use [**Stochastic Gradient Descent**](http://ufldl.stanford.edu/tutorial/supervised/OptimizationStochasticGradientDescent/) method for optimization, which updates the parameters $\theta$ as
$$\theta = \theta - \alpha \nabla_{\theta} L(\theta)$$
$\alpha$ is the learning rate. It can be easily implemented:
> `tf.train.GradientDescentOptimizer(alpha).minimize(loss)`

where alpha is the learning rate and loss is the loss function.

The whole training process can be implemented by TF as following.

In [ ]:

  W_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_neurons_1]))
  b_1 = tf.Variable(tf.zeros([num_neurons_1]))
  h_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)

  W_2 = tf.Variable(tf.truncated_normal([num_neurons_1, num_neurons_2]))
  b_2 = tf.Variable(tf.zeros([num_neurons_2]))  
  h_2 = tf.matmul(h_1, W_2) + b_2
    
  W_3 = tf.Variable(tf.truncated_normal([num_neurons_2, num_labels]))
  b_2 = tf.Variable(tf.zeros([num_labels]))
  o = tf.matmul(h_2, W_3) + b_3

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(o, y) +
    0.01*tf.nn.l2_loss(W_1) + 0.01*tf.nn.l2_loss(b_1) +
    0.01*tf.nn.l2_loss(W_2) + 0.01*tf.nn.l2_loss(b_2) +    
    0.01*tf.nn.l2_loss(W_3) + 0.01*tf.nn.l2_loss(b_3))

  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)


### 2. Convolutional neural network (CNN)

A Convolutional Neural Network (CNN) is comprised of several convolutional layers and subsampling layers, followed by fully connected multilayer neural network. The CNNs have been successfully applied to analyzing visual imagery 
￼

#### Convolutional layer
The input to a convolutional layer is a $m \times m \times r$ image where $m$ is the height and width of the image and $r$ is the number of channels.

The convolutional layer will have k patches (filters/kernels) of size $n \times n \times q$ where $n$ is smaller than the dimension of the image and $q$ is the depth of the patch and may vary for each patch. The size of the patch gives rise to the locally connected structure which are each convolved with the image to produce $k$ feature maps, $k = m-n+1$.

#### Pooling layer
We could compute the average (or max) value of a particular feature over a region of the image.
We call it **pooling** operation, the main pooling methods are Max pooling, Average pooling ...

Here we take a CNN with 2 convolutional layers (include pooling layer) followed by a neural network as an example.

<img src="figs/conv.png" style="zoom:50%">
<div align="center">*Example of a convolutional nexwork with 2 convolutional layers and 1 hidden neural layer.*</div>

The code for training by FT:

In [ ]:

  W_1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth_1], stddev=0.1))
  b_1 = tf.Variable(tf.zeros([depth_1]))

  W_2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth_1, depth_2], stddev=0.1))
  b_2 = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  W_3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  b_3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  W_4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  b_4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv_1 = tf.nn.conv2d(data, W_1, [1, 1, 1, 1], padding='SAME')
    c_1 = tf.nn.relu(conv_1 + b_1)
    pool_1 = tf.nn.max_pool(c_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv_2 = tf.nn.conv2d(pool_1, W_2, [1, 1, 1, 1], padding='SAME')
    c_2 = tf.nn.relu(conv_2 + b_2)
    pool_2 = tf.nn.max_pool(c_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(poo1_2, [shape[0], shape[1] * shape[2] * shape[3]])
    h = tf.nn.relu(tf.matmul(reshape, W_3) + b_3)
    dropout = tf.nn.dropout(h, 0.5)
    
    o = tf.matmul(dropout, W_4) + b_4
    return o
  
  # Training computation.
  logits = model(x)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y)+
    0.01*tf.nn.l2_loss(W_1) + 0.01*tf.nn.l2_loss(b_1) +
    0.01*tf.nn.l2_loss(W_2) + 0.01*tf.nn.l2_loss(b_2) +    
    0.01*tf.nn.l2_loss(W_3) + 0.01*tf.nn.l2_loss(b_3) +
    0.01*tf.nn.l2_loss(W_4) + 0.01*tf.nn.l2_loss(b_4))

    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(alpha).minimize(loss)
  

### 3. Recurrent neural network (RNN)

RNNs allow us to operate over sequential data (text, speech, handwritings, etc.), sequences as both the input and output.
RNNs are called recurrent because they perform the same task for every element of a sequence, with the output being depended on the previous computations. We can imagine that the networks have a “memory” which stores information about what has been calculated so far. 

#### Simple recurrent network
<img src="figs/srn.png" style="zoom:50%">
<div align="center">*Example of a simple recurrent network.*</div>

The unit of a simple recurrent network can be expressed as
- $h_t = \sigma_h(W_{xh}x_t+W_{hh}h_{t-1}+b_h)$
- $o_t = \sigma_o(W_{ho}h_t + b_y)$

in which $x_t$ is the $t$-th input vector, $h_t$ is the hidden layer and $o_t$ is the output vector; $W_{xh},W_{hh},W_{ho}$ are weight parameter matrices that are shared across all steps, $\sigma$ is the activation function.

#### Long short-term memory (LSTM)
A LSTM unit
<img src="figs/lstm.png" style="zoom:50%">
<div align="center">*A LSTM unit.*</div>

Two outputs from the LSTM unit
- $h_t = o_t * tanh(c_t)$
- $c_t = f_t * c_{t-1} + i_t*\tilde{c}_t$

in which, we have three gates and a memory cell
- Input gate: $i_t = \sigma(W_ix_t + U_ih_{t-1} + b_i)$
- Forget gate: $f_t = \sigma(W_fx_t + U_fh_{t-1} + b_f)$
- Output gate: $o_t = \sigma(W_ox_t + U_oh_{t-1} + b_o)$
- Memory cell: $\tilde{c}_t = \sigma(W_cx_t + U_ch_{t-1} + b_c)$

